## Test of dirichlet condition rotations in 2D curvilinear

To model numerical domains that do not align with the cartesian coordinate systems (e.g. deformed boundaries, annulus or spherical shell) the definition of the velocity field (for the stokes systems) often needs to be rotated to the surface normal and tangent basis of the model domain. As Underworld's discretisation assumes a linear cartesian coordinate system we can rotate bases vectors on non aligned surfaces, using the 
`CuvilinearDirichletCondition` and the `Curvilinear_Stokes`. This notebook tests this functionality in a cartestian domain.

Three driven models are:
1. 'control' model. Using standard `DirchletCondition` & `Stokes`. 
2. model with 'general' velocity field basis vectors  i.e. (e1, e2) aligned with (x,y) axis.
3. model with the velocity field bases 'fliped', i.e. (e1,  e2) align with (y, x) axis respectively.

By comparing these models we can test the algorithm for describing the velocity dirichlet conditions in rotated frames of reference.

N.B. In models `2` & `3` a proxy field for the cartesian velocity field `vc` is required to define nonlinear problems.

In [ ]:
import underworld as uw
import glucifer
import numpy as np
from underworld import function as fn

In [ ]:
# testing parameters
VTOL      = 1e-14
NONLINEAR = True

In [ ]:
# Brunch of utility python functions for visualisation and error testing

def display_vector(mesh, vec):
    '''
    displays the magnitude and vector arrows of the vector
    '''
    fig = glucifer.Figure()
    fig.append(glucifer.objects.Mesh(mesh, segmentsPerEdge=1))
    fig.append(glucifer.objects.Surface(mesh, fn.math.sqrt(fn.math.dot(vec,vec))))
    fig.append(glucifer.objects.VectorArrows(mesh, vec))
    fig.show()
    
def display_scalar(mesh, phi, *args, **kwargs):
    '''
    displays the magnitude and vector arrows of the vel
    '''
    fig = glucifer.Figure()
    fig.append(glucifer.objects.Mesh(mesh, segmentsPerEdge=1))
    fig.append(glucifer.objects.Surface(mesh, phi, **kwargs))
    fig.show()

def test_velocity(mesh, v0, v1, tol=VTOL):
    # L2 error btw velocity fields
    # v0 & v1 must be velocity MeshVariables ... todo
    err_fn = v0 - v1
    l2error_norm = np.sqrt(mesh.integrate( fn.math.pow(err_fn,2.) ))

    # Error checking. Does the velocity solutions look like the pure dirichlet condition case
    if uw.mpi.rank==0:
        print("L2 error norm for velocity", l2error_norm)
        if np.any(l2error_norm > tol):
            raise RuntimeError('Error are greater than tolerance of ', tol)

In [ ]:
## The numerical domain setup
# build mesh
mesh = uw.mesh.FeMesh_Cartesian(elementRes=(40,40), minCoord=(0.,0.,), maxCoord=(1.0,1.0)  )

# build fields
vField = mesh.add_variable(nodeDofCount=2)
pField = mesh.subMesh.add_variable(nodeDofCount=1)
tField = mesh.add_variable(nodeDofCount=1)

# definte wall and corner sets
N = mesh.specialSets["MaxJ_VertexSet"]; 
S = mesh.specialSets["MinJ_VertexSet"]
E = mesh.specialSets["MaxI_VertexSet"]; 
W = mesh.specialSets["MinI_VertexSet"]
allWalls = mesh.specialSets["AllWalls_VertexSet"]

# set static temperature condition, which proxies for the driving force of stokes
minR, maxR = mesh.minCoord[1], mesh.maxCoord[1]
deltaR     = maxR - minR
xcoord     = fn.coord()[0]
ycoord     = (fn.coord()[1] - minR) / deltaR
sinusoidalIC   = (1. - ycoord) + (0.2 * (fn.math.cos(xcoord * np.pi)) * fn.math.sin(ycoord * np.pi))
tField.data[:] = sinusoidalIC.evaluate(mesh)

In [ ]:
def solve_stokes( bc_conditions, viscosity ):
    '''
    Function to solve Stokes with either the Curvilinear stokes implementation 
    or standard Stokes implementation. 
    The choice is made depending on the input velocity condtions DirichletCondition or 
    '''
    
    
    if type(bc_conditions) == uw.conditions.DirichletCondition:
        
        
        stokesSLE = uw.systems.Stokes( vField, pField, 
                                   fn_viscosity = viscosity, 
                                   fn_bodyforce = (0.0,tField), 
                                   conditions   = bc_conditions, 
                                   _removeBCs   = False)
        stokesSolver = uw.systems.Solver(stokesSLE)
        stokesSolver.set_inner_method("lu")
        stokesSolver.solve(nonLinearIterate = True)
        
        
    elif type(rvBC) in [
                        uw.conditions.RotatedDirichletCondition, 
                        uw.conditions.CurvilinearDirichletCondition]:
        
        
        stokesSLE = uw.systems.Curvilinear_Stokes( vField, pField, 
                                           fn_viscosity = viscosity,
                                           fn_bodyforce = (0.0,tField),
                                           conditions   = rvBC, 
                                           _removeBCs   = False)
        
        stokesSolver = uw.systems.Solver(stokesSLE)
        stokesSolver.set_inner_method("lu")

        def postSolve():
            # realign solution using the rotation matrix on stokes
            uw.libUnderworld.Underworld.AXequalsY(
                stokesSLE._rot._cself,
                stokesSLE._velocitySol._cself,
                vcVec._cself,
                False
                )
            
        stokesSolver.solve(nonLinearIterate=True, callback_post_solve=postSolve)
        uw.libUnderworld.Underworld.AXequalsX( stokesSLE._rot._cself, stokesSLE._velocitySol._cself, False)

In [ ]:
# display_scalar(mesh, tField)

In [ ]:
# Model 1

vField.data[...] = 0.

vBC  = uw.conditions.DirichletCondition( variable=vField, indexSetsPerDof=(E+W,S+N))

viscosity = 1.0
if NONLINEAR:
    viscosity = 1. + 1e3*fn.math.pow(vField[0],2)
    
solve_stokes(vBC, viscosity)

standard_vField         = vField.copy()
standard_vField.data[:] = vField.data[:]

In [ ]:
# display_vector(vField.mesh, vField)

In [ ]:
# display_scalar(mesh, viscosity)

In [ ]:
# Model 2

vc       = vField.copy();
vc_eqNum = uw.systems.sle.EqNumber(vc, False)
vcVec    = uw.systems.sle.SolutionVector(vc, vc_eqNum)

if NONLINEAR:
    # turns on the nonlinearity by making viscosity a function of vField
    viscosity = 1.0 + 1e3*fn.math.pow(vc[0],2) + (0.*vField[0])
    # viscosity += (0. * vField[0]) ## Will not reproduce the same result because vField is used

vc.data[:] = vField.data[:] = 0. ### IMPORTANT reset the vField, starts nonlinear iterations at x_0 = 0

# define cartesian basis vectors
e_x = fn.misc.constant(1.0) * (1.,0.)
e_y = fn.misc.constant(1.0) * (0.,1.)

rvBC = uw.conditions.RotatedDirichletCondition( variable        = vField, 
                                                indexSetsPerDof = (E+W,S+N),
                                                basis_vectors   = (e_x, e_y) )

solve_stokes( rvBC, viscosity )

In [ ]:
# display_vector(vField.mesh, vField)

In [ ]:
# # visualise the difference
# diff = vc - standard_vField
# display_vector(vc.mesh, diff)

test_velocity(vc.mesh, vc, standard_vField)

In [ ]:
# Model 3

vField.data[...] = vc.data[...] = 0.

rvBC = uw.conditions.RotatedDirichletCondition( variable        = vField, 
                                                indexSetsPerDof = (S+N,E+W),
                                                basis_vectors   = (e_y, e_x) )

solve_stokes( rvBC, viscosity )

In [ ]:
# display_vector(vc.mesh, vc)

In [ ]:
# # visualise the difference
# diff = vc - standard_vField
# display_vector(vc.mesh, diff)

test_velocity(vc.mesh, vc, standard_vField)